In [ ]:
#Uncomment if you need to install these packages
#pip install pslpython
#pip install ipywidgets

In [28]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  4 08:04:34 2022

@author: ewanhilton
"""
import os

from pslpython.model import Model
from pslpython.partition import Partition
from pslpython.predicate import Predicate
from pslpython.rule import Rule
from classes.Dataset import get_dataset
from classes.ANYBurlToPSLConverter import ANYBurlToPSLConverter
from classes.RuleImporter import RuleImporter
from classes.EntityConverter import EntityConverter
from classes.DatasetGenerator import encode_text
from tqdm import tqdm
from datetime import datetime
from classes.PSLFileBuilder import build_target_files
from classes.DatasetGenerator import encode_text
from classes.PSLFileBuilder import PSLFileBuilder
from classes.CustomRuleWeighting import custom_rule_weighting
from operator import itemgetter
import glob
import json

MODEL_NAME = 'ANYBurl and PSL Model'

DATA_DIR = os.path.join('data')

ADDITIONAL_PSL_OPTIONS = {
    'log4j.threshold': 'INFO'
}

ADDITIONAL_CLI_OPTIONS = [
    # '--postgres'
]

#Load hyper parameters from json
f = open('params.json')
params = json.load(f)
f.close()

MAX_RULES = None if params['PSL']['MAX_RULES'] == "None" else int(params['PSL']['MAX_RULES'])
ANYBURL_RULES_THRESHOLD = float(params['PSL']['ANYBURL_RULES_THRESHOLD'])
PREDICT_PREDICATES_ONE_AT_A_TIME = params['PSL']['PREDICT_PREDICATES_ONE_AT_A_TIME'] == "True"
TOP_K_RULES_ENABLED = params['PSL']['TOP_K_RULES']['ENABLED'] == "True"
TOP_K_RULES_K = int(params['PSL']['TOP_K_RULES']['K'])
CUSTOM_RULE_WEIGHTING_ENABLED = params['PSL']['CUSTOM_RULE_WEIGHTING']['ENABLED'] == "True"
CUSTOM_RULE_WEIGHTING_TYPE = params['PSL']['CUSTOM_RULE_WEIGHTING']['WEIGHTING_TYPE']
RULE_LENGTH_ENABLED = params['PSL']['RULE_LENGTH']['ENABLED'] == "True"
RULE_LENGTH = int(params['PSL']['RULE_LENGTH']['LENGTH'])

def main():
    print(f"PSL run stated at {get_date_time()}")
    importer = RuleImporter()
    importer.import_rules()
   
    dataset = get_dataset()
    entity_converter = EntityConverter(dataset)

    train_triples = dataset.training.mapped_triples.numpy()
    val_triples = dataset.validation.mapped_triples.numpy()
    #test_triples = dataset.testing.mapped_triples.numpy()  
    
    #Delete all pre-existing predicates from the folder
    for f in glob.glob("inferred-predicates/*"):
                    os.remove(f)
            
    if PREDICT_PREDICATES_ONE_AT_A_TIME :
        for relindex,name in tqdm(entity_converter.relindex_to_name.items()): 
            try:
                model = Model(MODEL_NAME)
                
                #Delete existing targets files so we only deal with one at a time
                for f in glob.glob("data/targets/*"):
                    os.remove(f)
                    
                try:
                    os.remove("/tmp/psl_h2_ewan@fidra.mv.db")
                except:
                    print("File does not exist")
                
                #Add Target Files
                build_target_files(name,relindex, entity_converter, train_triples)
                # Add Predicates
                add_predicates(model,entity_converter,name)

                # Add Rules
                add_rules(model,importer.rules, encode_text(name), name)

                # Add Data
                add_data(model, entity_converter, name)

                # Inference
                results = infer(model)
                write_results(results, model)
            except Exception as e: 
                print(e)
                continue
    else:
        name = "All Predicates"
        model = Model(MODEL_NAME)

        #Add Target Files
        filebuilder = PSLFileBuilder(train_triples, val_triples, entity_converter)
        filebuilder.build_target_files()
        # Add Predicates
        add_predicates(model,entity_converter)

        # Add Rules
        add_rules(model,importer.rules, None)

        # Add Data
        add_data(model, entity_converter)
        
        del dataset
        del importer
        del train_triples
        del val_triples
        
        # Inference
        results = infer(model)
        write_results(results, model)
    print(f"PSL run ended at {get_date_time()}")

def get_top_k_rules(original_rules,k=10):
    converter = ANYBurlToPSLConverter(original_rules)
    rules_and_weights = []
    
    if RULE_LENGTH_ENABLED:
        for converted_rule in converter.get_rules_by_length(RULE_LENGTH):
            rules_and_weights.append([converted_rule,converted_rule.split(':')[0]]) 
    else:
        for converted_rule in converter.converted_rules:
            rules_and_weights.append([converted_rule,converted_rule.split(':')[0]])       
    return sorted(rules_and_weights, key=itemgetter(1),reverse=True)[:k]    
    
def add_predicates(model,entity_converter,target_name=None):
    if target_name is not None:
        print(f"Adding predicates for {target_name}")
    else:
        print("Adding predicates for All Predicates")
    for relindex,name in tqdm(entity_converter.relindex_to_name.items()):
        is_closed = False
        if target_name is not None:
            is_closed = name != target_name
        predicate = Predicate(encode_text(name), closed = is_closed, size = 2)
        model.add_predicate(predicate)

def add_rules(model, rules, name, target_name=None):
    if target_name is not None:
        print(f"Adding rules for {target_name}")
    else:
        print("Adding rules for All Predicates")
    
    total_rules = 0
        
    if TOP_K_RULES_ENABLED == True:
        print(f"Adding up to {TOP_K_RULES_K} Top K Rules")
        for rule in get_top_k_rules(rules, TOP_K_RULES_K):
            if name is None or rule.split('->')[1].split('(')[0].replace(' ','') == name:
                if MAX_RULES != None and total_rules >= MAX_RULES:
                    print(f"Maximum number of rules added ({total_rules} rules added)")
                    return
                if float(rule[1]) > ANYBURL_RULES_THRESHOLD:
                    
                    if CUSTOM_RULE_WEIGHTING_ENABLED:
                        model.add_rule(Rule(custom_rule_weighting(rule[0],weighting_type = CUSTOM_RULE_WEIGHTING_TYPE)))
                    else:
                        model.add_rule(Rule(rule[0]))
                    total_rules += 1 
    else:      
        converter = ANYBurlToPSLConverter(rules)
        for rule in tqdm(converter.converted_rules if not RULE_LENGTH_ENABLED else converter.get_rules_by_length(RULE_LENGTH):):
            if name is None or rule.split('->')[1].split('(')[0].replace(' ','') == name:
                if MAX_RULES != None and total_rules >= MAX_RULES:
                    print(f"Maximum number of rules added ({total_rules} rules added)")
                    return
                if float(rule.split(':')[0]) > ANYBURL_RULES_THRESHOLD:
                    if CUSTOM_RULE_WEIGHTING_ENABLED:
                        model.add_rule(Rule(custom_rule_weighting(rule,weighting_type = CUSTOM_RULE_WEIGHTING_TYPE)))
                    else:
                        model.add_rule(Rule(rule))
                    total_rules += 1
                    continue
                
    if total_rules < 1:
        raise Exception(f"Failed to add any rules. Aborting inference for {target_name}")
    print(f"{total_rules} rules added")
    
def add_data(model,entity_converter, target_name = None):
    if target_name is not None:
        print(f"Adding data for {target_name}")
    else:
        print("Adding data for All Predicates")
        
    for relindex,name in tqdm(entity_converter.relindex_to_name.items()):        
        path = f'data/obs/{encode_text(name)}_obs.txt'
        if path_exists(path): #Check file has content before adding  
            model.get_predicate(encode_text(name)).add_data_file(Partition.OBSERVATIONS, path)

        path =  f'data/targets/{encode_text(name)}_targets.txt'
        if path_exists(path):
            model.get_predicate(encode_text(name)).add_data_file(Partition.TARGETS, path)

        path = f'data/truth/{encode_text(name)}_truth.txt'
        if path_exists(path):
            model.get_predicate(encode_text(name)).add_data_file(Partition.TRUTH, path)
   
def path_exists(path):
    try:
        return os.path.getsize(f"{path}") > 0
    except:
        return False

def infer(model): 
    print(f"Inference starting at {get_date_time()}")
    return model.infer(psl_config = ADDITIONAL_PSL_OPTIONS)
     
def get_date_time():
    return f"{str(datetime.now().time()).split('.')[0]} on {datetime.today().strftime('%d-%b-%Y')}"

def write_results(results, model):
    print(f"Inferenced completed at {get_date_time()}")
    out_dir = 'inferred-predicates'
    os.makedirs(out_dir, exist_ok = True)
    print("Writing predicates")
    for predicate in tqdm(model.get_predicates().values()):
        if (predicate.closed()):
            continue       
        try:
            out_path = os.path.join(out_dir, "%s.txt" % (predicate.name()))
            results[predicate].to_csv(out_path, sep = "\t", header = False, index = False)     
        except:
            continue

In [ ]:
main()